In [1]:
# Librerías
import numpy as np
import pandas as pd
import os
import gc
import re
import requests

In [2]:
gc.collect()

0

In [3]:
def escribir_matriz(matriz, archivo):
    with open(archivo, 'w') as file:
        file.write(f"{len(matriz)} {len(matriz[0])}\n\n")
        for fila in matriz:
            fila_str = ' '.join(str(elemento) for elemento in fila)
            file.write(fila_str + '\n')

In [4]:
dict_estaciones = {'Tag ID defectuosa': "150", 'Tag ID defectuosa NS 1649 (Kiosco478)': '478', 'kiosco100-FallaRTC': '100', 'CE-642': '642', 'CE-622': '622', 'Kiosco 150177': '150'}

def procesar_estacion(valor):
    # Si el valor es un número entero, lo dejamos igual
    try:
        return int(valor)
    except ValueError:
        pass

    # Si el valor tiene el formato "%d-%d", obtenemos el primer número
    if re.match(r'^\d+-\d+$', valor):
        return int(valor.split('-')[0])

    # Si el valor está en el diccionario, usamos el valor corregido del diccionario
    if valor in dict_estaciones:
        return int(dict_estaciones[valor])

    # Si no está en el diccionario, pedimos manualmente la estación
    print(f"Valor inesperado: {valor}")
    nuevo_valor = input("Ingresa manualmente la estación correcta: ")
    return int(nuevo_valor)

In [5]:
datos_2021 = pd.read_csv('ecobici/2021.csv')
datos_2021['Ciclo_Estacion_Retiro'] = datos_2021['Ciclo_Estacion_Retiro'].apply(procesar_estacion)
datos_2021['Ciclo_Estacion_Arribo'] = datos_2021['Ciclo_Estacion_Arribo'].apply(procesar_estacion)

datos_2021.to_csv('ecobici/2021.csv', index=False)
del datos_2021

datos_2022 = pd.read_csv('ecobici/2022.csv')
datos_2022['Ciclo_Estacion_Retiro'] = datos_2022['Ciclo_Estacion_Retiro'].apply(procesar_estacion)
datos_2022['Ciclo_Estacion_Arribo'] = datos_2022['Ciclo_Estacion_Arribo'].apply(procesar_estacion)

datos_2022.to_csv('ecobici/2022.csv', index=False)
del datos_2022

datos_2024 = pd.read_csv('ecobici/2024.csv')
datos_2024['Ciclo_Estacion_Retiro'] = datos_2024['Ciclo_Estacion_Retiro'].apply(procesar_estacion)
datos_2024['Ciclo_Estacion_Arribo'] = datos_2024['Ciclo_Estacion_Arribo'].apply(procesar_estacion)

datos_2024.to_csv('ecobici/2024.csv', index=False)
del datos_2024

/tmp/ipykernel_34539/4104760429.py:8: DtypeWarning: Columns (0,3,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  datos_2022 = pd.read_csv('ecobici/2022.csv')


In [33]:
data_prueba = pd.read_csv("mibici/2019.csv", low_memory=False)

In [34]:
data_prueba

,Viaje_Id,Usuario_Id,Genero,Año_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Tiempo_viaje
0,8901295,341162,M,1992.0,2019-01-01 00:00:00,2019-01-01 00:16:09,50,54,16.150000
1,8901296,413443,M,1985.0,2019-01-01 06:01:05,2019-01-01 06:16:27,38,194,15.366667
2,8901297,444232,M,1995.0,2019-01-01 06:01:19,2019-01-01 06:09:55,86,18,8.600000
3,8901298,437533,M,1998.0,2019-01-01 06:01:25,2019-01-01 06:02:04,86,86,0.650000
4,8901299,430224,M,1992.0,2019-01-01 06:01:57,2019-01-01 06:05:01,27,52,3.066667
...,...,...,...,...,...,...,...,...,...
4631214,14420211,545173,M,1997.0,2019-12-31 23:57:29,2020-01-01 00:16:00,51,52,18.516667
4631215,14420212,162325,M,1990.0,2019-12-31 23:57:54,2020-01-01 00:05:41,54,163,7.783333
4631216,14420213,71721,M,1987.0,2019-12-31 23:57:56,2020-01-01 00:06:28,81,42,8.533333
4631217,14420214,117554,M,1982.0,2019-12-31 23:59:14,2020-01-01 00:19:47,51,8,20.550000


In [4]:
years = [2019, 2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"Procesando año {year}")
    try:
        current_data = pd.read_csv(f"ecobici/{year}.csv", low_memory=False)
    except UnicodeDecodeError:
        current_data = pd.read_csv(f"ecobici/{year}.csv", low_memory=False, encoding='latin1')

    estaciones = np.sort(current_data['Ciclo_Estacion_Retiro'].unique())
    estaciones = estaciones.astype(float).astype(int)
    n_estaciones = len(estaciones)
    
    matriz_estaciones = np.column_stack((estaciones, range(n_estaciones)))
    diccionario_estaciones = {estacion: i for i, estacion in enumerate(estaciones)}
    
    folder_name = f"matrices_estaciones/estaciones_{year}.txt"
    os.makedirs(os.path.dirname(folder_name), exist_ok=True)
    escribir_matriz(matriz_estaciones, folder_name)
    print(f"Matriz de estaciones guardada para el año {year}")

    dates = current_data['Fecha_Retiro'].unique()
    
    for date in dates:
        print(f"Procesando fecha: {date}")
        adj_matrix = np.zeros((n_estaciones, n_estaciones))
        current_date_data = current_data[current_data['Fecha_Retiro'] == date]
        
        for _, d in current_date_data.iterrows():
            estacion_origen = d['Ciclo_Estacion_Retiro']
            estacion_destino = d['Ciclo_Estacion_Arribo']
            
            if estacion_origen in diccionario_estaciones and estacion_destino in diccionario_estaciones:
                adj_matrix[diccionario_estaciones[estacion_origen]][diccionario_estaciones[estacion_destino]] += 1
                
        date_clean = date.replace('/', '_')
        # Crear el archivo con la fecha modificada
        output_file = f"matrices_{year}/{date_clean}.txt"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        escribir_matriz(adj_matrix, output_file)
        
    gc.collect()  # Forzamos la recolección de basura si es necesario.

print("Proceso terminado")
requests.post("https://ntfy.sh/Compu_CIMAT", data="Terminaron las matrices".encode(encoding='utf-8'))


Procesando año 2019
Matriz de estaciones guardada para el año 2019
Procesando fecha: 01/01/2019
Procesando fecha: 31/12/2018
Procesando fecha: 02/01/2019
Procesando fecha: 03/01/2019
Procesando fecha: 04/01/2019
Procesando fecha: 05/01/2019
Procesando fecha: 06/01/2019
Procesando fecha: 07/01/2019
Procesando fecha: 08/01/2019
Procesando fecha: 09/01/2019
Procesando fecha: 10/01/2019
Procesando fecha: 11/01/2019
Procesando fecha: 12/01/2019
Procesando fecha: 13/01/2019
Procesando fecha: 14/01/2019
Procesando fecha: 15/01/2019
Procesando fecha: 16/01/2019
Procesando fecha: 17/01/2019
Procesando fecha: 18/01/2019
Procesando fecha: 19/01/2019
Procesando fecha: 20/01/2019
Procesando fecha: 21/01/2019
Procesando fecha: 22/01/2019
Procesando fecha: 23/01/2019
Procesando fecha: 24/01/2019
Procesando fecha: 25/01/2019
Procesando fecha: 26/01/2019
Procesando fecha: 27/01/2019
Procesando fecha: 28/01/2019
Procesando fecha: 29/01/2019
Procesando fecha: 30/01/2019
Procesando fecha: 31/01/2019
Proce

<Response [200]>

In [4]:
years = [2023]#, 2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"Procesando año {year}")
    try:
        current_data = pd.read_csv(f"mibici/{year}.csv", low_memory=False)
    except UnicodeDecodeError:
        current_data = pd.read_csv(f"mibici/{year}.csv", low_memory=False, encoding='latin1')

    estaciones = np.sort(current_data['Origen_Id'].unique())
    estaciones = estaciones.astype(int)
    n_estaciones = len(estaciones)
    
    matriz_estaciones = np.column_stack((estaciones, range(n_estaciones)))
    diccionario_estaciones = {estacion: i for i, estacion in enumerate(estaciones)}
    
    folder_name = f"matrices_estaciones/estaciones_{year}.txt"
    os.makedirs(os.path.dirname(folder_name), exist_ok=True)
    escribir_matriz(matriz_estaciones, folder_name)
    print(f"Matriz de estaciones guardada para el año {year}")

    current_data['Fecha'] = pd.to_datetime(current_data['Inicio_del_viaje']).dt.strftime('%d-%m-%Y')
    dates = current_data['Fecha'].unique()
    
    for date in dates:
        print(f"Procesando fecha: {date}")
        adj_matrix = np.zeros((n_estaciones, n_estaciones))
        current_date_data = current_data[current_data['Fecha'] == date]
        
        for _, d in current_date_data.iterrows():
            estacion_origen = d['Origen_Id']
            estacion_destino = d['Destino_Id']
            
            if estacion_origen in diccionario_estaciones and estacion_destino in diccionario_estaciones:
                adj_matrix[diccionario_estaciones[estacion_origen]][diccionario_estaciones[estacion_destino]] += 1
                
        date_clean = date.replace('-', '_')
        # Crear el archivo con la fecha modificada
        output_file = f"matrices_{year}/{date_clean}.txt"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        escribir_matriz(adj_matrix, output_file)
        
    gc.collect()  # Forzamos la recolección de basura si es necesario.

print("Proceso terminado")

Procesando año 2023
Matriz de estaciones guardada para el año 2023
Procesando fecha: 01-01-2023
Procesando fecha: 02-01-2023
Procesando fecha: 03-01-2023
Procesando fecha: 04-01-2023
Procesando fecha: 05-01-2023
Procesando fecha: 06-01-2023
Procesando fecha: 07-01-2023
Procesando fecha: 08-01-2023
Procesando fecha: 09-01-2023
Procesando fecha: 10-01-2023
Procesando fecha: 11-01-2023
Procesando fecha: 12-01-2023
Procesando fecha: 13-01-2023
Procesando fecha: 14-01-2023
Procesando fecha: 15-01-2023
Procesando fecha: 16-01-2023
Procesando fecha: 17-01-2023
Procesando fecha: 18-01-2023
Procesando fecha: 19-01-2023
Procesando fecha: 20-01-2023
Procesando fecha: 21-01-2023
Procesando fecha: 22-01-2023
Procesando fecha: 23-01-2023
Procesando fecha: 24-01-2023
Procesando fecha: 25-01-2023
Procesando fecha: 26-01-2023
Procesando fecha: 27-01-2023
Procesando fecha: 28-01-2023
Procesando fecha: 29-01-2023
Procesando fecha: 30-01-2023
Procesando fecha: 31-01-2023
Procesando fecha: 01-02-2023
Proce

In [5]:
import requests
requests.post("https://ntfy.sh/Compu_CIMAT", data="Terminaron las matrices".encode(encoding='utf-8'))

<Response [200]>

In [20]:
data_prueba = pd.read_csv("ecobici/2024.csv", low_memory=False)
data_prueba

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo,Fecha_Arribo,Tiempo_viaje
0,M,45.0,2578060,304,31/03/2024,23:25:50,117,NaN,00:00:04,01/04/2024,34.233333
1,M,27.0,2743490,104,31/03/2024,23:42:47,059,NaN,00:00:17,01/04/2024,17.500000
2,M,30.0,3003159,488,31/03/2024,23:52:14,534,NaN,00:00:18,01/04/2024,8.066667
3,M,48.0,5355870,152,31/03/2024,23:50:23,566,NaN,00:00:22,01/04/2024,9.983333
4,F,23.0,8602260,003,31/03/2024,23:22:30,465,NaN,00:00:24,01/04/2024,37.900000
...,...,...,...,...,...,...,...,...,...,...,...
1958588,M,32.0,7259187,053,30/04/2024,22:51:57,053,NaN,23:59:44,30/04/2024,67.783333
1958589,M,20.0,4030029,362,30/04/2024,23:27:50,688,NaN,23:59:45,30/04/2024,31.916667
1958590,M,21.0,4427936,180,30/04/2024,23:54:44,068,NaN,23:59:52,30/04/2024,5.133333
1958591,M,29.0,7050396,029,30/04/2024,23:53:29,055,NaN,23:59:55,30/04/2024,6.433333


In [21]:
data_prueba.rename(columns={"Ciclo_EstacionArribo": "Ciclo_Estacion_Arribo", "Fecha Arribo": "Fecha_Arribo"}, inplace=True)

In [7]:
data_prueba.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Hora_Retiro
0,M,45.0,3297082,150,31/12/2022,23:58:00,040,01/01/2023,00:06:24,2022-12-31 23:58:00
1,F,27.0,8922254,214,31/12/2022,23:36:34,025,01/01/2023,00:07:08,2022-12-31 23:36:34
2,M,34.0,2320075,214,31/12/2022,23:36:33,025,01/01/2023,00:07:12,2022-12-31 23:36:33
3,M,58.0,2096493,137,31/12/2022,23:57:19,036,01/01/2023,00:07:48,2022-12-31 23:57:19
4,M,23.0,6294433,260,31/12/2022,23:57:42,043,01/01/2023,00:09:33,2022-12-31 23:57:42


In [22]:
estaciones_con_letras = data_prueba[data_prueba['Ciclo_Estacion_Retiro'].str.contains(r'[a-zA-Z]', na=False)]

# Mostrar las estaciones que contienen letras
print(estaciones_con_letras['Ciclo_Estacion_Retiro'].unique())


[]


In [23]:
estaciones_con_letras = data_prueba[data_prueba['Ciclo_Estacion_Retiro'].str.contains('-', na=False)]

# Mostrar las estaciones que contienen letras
print(estaciones_con_letras['Ciclo_Estacion_Retiro'].unique())

['158-159' '271-272' '107-108' '192-193' '237-238' '268-269' '264-275'
 '445-446' '390-391' '266-267' '273-274' '235-236']


In [24]:
estaciones_con_letras = data_prueba[data_prueba['Ciclo_Estacion_Arribo'].str.contains(r'[a-zA-Z]', na=False)]

# Mostrar las estaciones que contienen letras
print(estaciones_con_letras['Ciclo_Estacion_Arribo'].unique())

[]


In [10]:
dict_estaciones = {'Tag ID defectuosa': "150", 'Tag ID defectuosa NS 1649 (Kiosco478)': '478', 'kiosco100-FallaRTC': '100', 'CE-642': '642', 'CE-622': '622', 'Kiosco 150177': '150'}

In [25]:
print("Estoy en las estaciones de retiro")
data_prueba['Ciclo_Estacion_Retiro'] = data_prueba['Ciclo_Estacion_Retiro'].apply(procesar_estacion)
print("Estoy en las estaciones de arribo")
data_prueba['Ciclo_Estacion_Arribo'] = data_prueba['Ciclo_Estacion_Arribo'].apply(procesar_estacion)

Estoy en las estaciones de retiro
Estoy en las estaciones de arribo


In [26]:
output_file = "ecobici/2024.csv"  # Aquí pones el nombre original del archivo

# Guardar el DataFrame actualizado en el archivo CSV original
data_prueba.to_csv(output_file, index=False, escapechar='\\')

print(f"Archivo sobrescrito exitosamente en {output_file}")

Archivo sobrescrito exitosamente en ecobici/2024.csv


In [30]:
def calcular_tiempo_viaje2(df):
    # Intentar convertir las columnas Inicio_del_viaje y Fin_del_viaje a datetime, eliminando errores
    try:
        df['Inicio_del_viaje'] = pd.to_datetime(df['Inicio_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df['Fin_del_viaje'] = pd.to_datetime(df['Fin_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    except Exception as e:
        print("Error al convertir las columnas de fecha y hora:", e)
    
    # Eliminar filas con datos inválidos (NaT) en las columnas de fechas
    df = df.dropna(subset=['Inicio_del_viaje', 'Fin_del_viaje'])

    # Verificar si hay datos después de eliminar NaT
    if df.empty:
        print("Todas las filas fueron eliminadas por datos inválidos.")
        return df

    # Calcular la diferencia en minutos usando .loc para evitar el warning
    df.loc[:, 'Tiempo_viaje'] = (df['Fin_del_viaje'] - df['Inicio_del_viaje']).dt.total_seconds() / 60

    # Filtrar los viajes que duren más de 30 minutos
    df = df[df['Tiempo_viaje'] <= 90].copy()

    # Eliminar las columnas temporales
    #df = df.drop(columns=['Inicio_del_viaje', 'Fin_del_viaje'])

    return df

In [31]:
def cargar_datos(direccion):
    df_list = []
    year = direccion.split('/')[-2]
    if year == '2024':
        lim = 8
    else:
        lim = 13
    
    for file in range(1, lim):
        if file < 10:
            file_path = direccion + year + "_" + f'0{file}.csv'
        else:
            file_path = direccion + year + "_" + f'{file}.csv'
        try:
            df = pd.read_csv(file_path, low_memory=False, encoding='utf-8')
        except UnicodeDecodeError:
            print(f"Problema de codificación en {file_path}, intentando con 'latin1'")
            df = pd.read_csv(file_path, low_memory=False, encoding='latin1')
        
        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

In [32]:
direccion = 'mibici/'

years = ["2019", "2020", "2021","2022","2023","2024"]

for year in years:
    df = cargar_datos(direccion + year + '/')
    if 'Unnamed: 9' in df.columns:
        df = df.drop(columns='Unnamed: 9')
    df = calcular_tiempo_viaje2(df)
    df.to_csv(direccion + year + '.csv', index=False)
    del df
    print(f'Los datos de {year} han sido limpiados y guardados')

Los datos de 2019 han sido limpiados y guardados
Los datos de 2020 han sido limpiados y guardados
Problema de codificación en mibici/2021/2021_06.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_07.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_08.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_09.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_10.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_11.csv, intentando con 'latin1'
Problema de codificación en mibici/2021/2021_12.csv, intentando con 'latin1'
Los datos de 2021 han sido limpiados y guardados
Problema de codificación en mibici/2022/2022_01.csv, intentando con 'latin1'
Problema de codificación en mibici/2022/2022_02.csv, intentando con 'latin1'
Problema de codificación en mibici/2022/2022_03.csv, intentando con 'latin1'
Problema de codificación en mibici/2022/2022_04.csv, intentando con 'latin1'
Proble

/tmp/ipykernel_55307/1468907596.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Tiempo_viaje'] = (df['Fin_del_viaje'] - df['Inicio_del_viaje']).dt.total_seconds() / 60


Los datos de 2023 han sido limpiados y guardados
Problema de codificación en mibici/2024/2024_01.csv, intentando con 'latin1'
Problema de codificación en mibici/2024/2024_02.csv, intentando con 'latin1'
Problema de codificación en mibici/2024/2024_03.csv, intentando con 'latin1'
Problema de codificación en mibici/2024/2024_04.csv, intentando con 'latin1'
Problema de codificación en mibici/2024/2024_05.csv, intentando con 'latin1'
Problema de codificación en mibici/2024/2024_06.csv, intentando con 'latin1'
Los datos de 2024 han sido limpiados y guardados
